In [1]:
from gridfm_datakit.network import load_net_from_pglib
from gridfm_datakit.process.process_network import (
    init_julia,
    pf_preprocessing,
    pf_post_processing,
)
from gridfm_datakit.process.solvers import run_opf, run_pf
import pandas as pd
import numpy as np
from gridfm_datakit.utils.column_names import (
    BUS_COLUMNS,
    GEN_COLUMNS,
    BRANCH_COLUMNS,
)

jl = init_julia(max_iter=200)


def extract_results(processed_data, n_buses):
    bus_columns = BUS_COLUMNS
    bus_data = np.concatenate([item[0] for item in processed_data], axis=0)
    df_bus = pd.DataFrame(bus_data, columns=bus_columns)

    gen_data = np.concatenate([item[1] for item in processed_data], axis=0)
    df_gen = pd.DataFrame(
        gen_data,
        columns=GEN_COLUMNS,
    )

    branch_data = np.concatenate([item[2] for item in processed_data], axis=0)
    df_branch = pd.DataFrame(
        branch_data,
        columns=BRANCH_COLUMNS,
    )

    return df_bus, df_gen, df_branch

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [ ]:
case_name = "case24_ieee_rts"


# Load network
net = load_net_from_pglib(case_name)
n_buses = net.buses.shape[0]

print(
    f"  Loaded {case_name}: {net.buses.shape[0]} buses, {net.gens.shape[0]} gens, {net.branches.shape[0]} branches",
)

# Step 1: Run OPF
print("  Running OPF...")
opf_result = run_opf(net, jl)
# Step 2: set gen setpoints with OPF results
net = pf_preprocessing(net, opf_result)
# Step 3: Run PF
print("  Running PF...")
pf_result = run_pf(net, jl, fast=True)
# Step 4: format results
processed_data = pf_post_processing(
    0, net, res=opf_result, res_dc=None, include_dc_res=False
)
processed_data = [
    (
        processed_data["bus"],
        processed_data["gen"],
        processed_data["branch"],
        processed_data["Y_bus"],
        processed_data["runtime"],
    )
]
# Step 5: save results
node_path = "node_data"
branch_path = "branch_data"
gen_path = "gen_data"
y_bus_path = "y_bus_data"
runtime_path = "runtime_data"
# Step 5: get dataframes
df_bus, df_gen, df_branch = extract_results(processed_data, n_buses)

  Loaded case24_ieee_rts: 24 buses, 33 gens, 38 branches
  Running OPF...
  Running PF...


In [4]:
df_bus

,bus,Pd,Qd,Pg,Qg,Vm,Va,PQ,PV,REF,vn_kv,min_vm_pu,max_vm_pu,GS,BS
0,0.0,108.0,22.0,184.000000,6.572191,1.047228,-6.755434e+00,0.0,1.0,0.0,138.0,0.95,1.05,0.0,0.0
1,1.0,97.0,20.0,184.000000,3.780734,1.047206,-6.831083e+00,0.0,1.0,0.0,138.0,0.95,1.05,0.0,0.0
2,2.0,180.0,37.0,0.000000,0.000000,1.013779,-6.187465e+00,1.0,0.0,0.0,138.0,0.95,1.05,0.0,0.0
3,3.0,74.0,15.0,0.000000,0.000000,1.016249,-9.574331e+00,1.0,0.0,0.0,138.0,0.95,1.05,0.0,0.0
4,4.0,71.0,14.0,0.000000,0.000000,1.035427,-9.737554e+00,1.0,0.0,0.0,138.0,0.95,1.05,0.0,0.0
5,5.0,136.0,28.0,0.000000,0.000000,1.032406,-1.231055e+01,1.0,0.0,0.0,138.0,0.95,1.05,0.0,-1.0
6,6.0,125.0,25.0,211.028282,49.315453,1.036005,-9.876561e+00,0.0,1.0,0.0,138.0,0.95,1.05,0.0,0.0
7,7.0,171.0,35.0,0.000000,0.000000,1.008963,-1.255316e+01,1.0,0.0,0.0,138.0,0.95,1.05,0.0,0.0
8,8.0,175.0,36.0,0.000000,0.000000,1.023927,-7.818243e+00,1.0,0.0,0.0,138.0,0.95,1.05,0.0,0.0
9,9.0,195.0,40.0,0.000000,0.000000,1.050000,-9.678324e+00,1.0,0.0,0.0,138.0,0.95,1.05,0.0,0.0


In [5]:
df_gen

,idx,bus,p_mw,q_mvar,min_p_mw,max_p_mw,min_q_mvar,max_q_mvar,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,in_service,is_slack_gen
0,0.0,0.0,15.999999,4.862180,16.0,20.0,0.0,10.0,0.000000,130.0000,400.6849,1.0,0.0
1,1.0,0.0,15.999999,4.862180,16.0,20.0,0.0,10.0,0.000000,130.0000,400.6849,1.0,0.0
2,2.0,0.0,76.000001,-1.576085,15.2,76.0,-25.0,30.0,0.014142,16.0811,212.3076,1.0,0.0
3,3.0,0.0,76.000001,-1.576085,15.2,76.0,-25.0,30.0,0.014142,16.0811,212.3076,1.0,0.0
4,4.0,1.0,15.999999,4.813675,16.0,20.0,0.0,10.0,0.000000,130.0000,400.6849,1.0,0.0
5,5.0,1.0,15.999999,4.813675,16.0,20.0,0.0,10.0,0.000000,130.0000,400.6849,1.0,0.0
6,6.0,1.0,76.000001,-2.923308,15.2,76.0,-25.0,30.0,0.014142,16.0811,212.3076,1.0,0.0
7,7.0,1.0,76.000001,-2.923308,15.2,76.0,-25.0,30.0,0.014142,16.0811,212.3076,1.0,0.0
8,8.0,6.0,70.342761,16.438484,25.0,100.0,0.0,60.0,0.052672,43.6615,781.5210,1.0,0.0
9,9.0,6.0,70.342761,16.438484,25.0,100.0,0.0,60.0,0.052672,43.6615,781.5210,1.0,0.0


In [6]:
df_branch

,idx,from_bus,to_bus,pf,qf,pt,qt,r,x,b,...,Ytf_r,Ytf_i,Ytt_r,Ytt_i,tap,shift,ang_min,ang_max,rate_a,br_status
0,0.0,0.0,1.0,10.094910,-27.005262,-10.092423,-23.548590,0.0026,0.0139,0.4611,...,-13.001950,69.510427,13.001950,-69.279877,1.00,0.0,-30.0,30.0,175.0,1.0
1,1.0,0.0,2.0,-0.645733,13.640363,0.786072,-19.173397,0.0546,0.2112,0.0572,...,-1.147382,4.438224,1.147382,-4.409624,1.00,0.0,-30.0,30.0,175.0,1.0
2,2.0,0.0,4.0,66.550823,-2.062911,-65.670292,2.992711,0.0218,0.0845,0.0229,...,-2.862587,11.095806,2.862587,-11.084356,1.00,0.0,-30.0,30.0,175.0,1.0
3,3.0,1.0,3.0,44.117091,13.248048,-43.466499,-14.386867,0.0328,0.1267,0.0343,...,-1.914911,7.396929,1.914911,-7.379779,1.00,0.0,-30.0,30.0,175.0,1.0
4,4.0,1.0,5.0,52.975332,-5.918724,-51.699208,5.226113,0.0497,0.1920,0.0520,...,-1.263535,4.881262,1.263535,-4.855262,1.00,0.0,-30.0,30.0,175.0,1.0
5,5.0,2.0,8.0,21.254531,-15.447360,-21.062136,12.848065,0.0308,0.1190,0.0322,...,-2.038434,7.875767,2.038434,-7.859667,1.00,0.0,-30.0,30.0,175.0,1.0
6,6.0,2.0,23.0,-202.040603,-2.379244,203.009892,37.737204,0.0023,0.0839,0.0000,...,-0.316986,11.563107,0.326496,-11.910001,1.03,0.0,-30.0,30.0,400.0,1.0
7,7.0,3.0,8.0,-30.533501,-0.613133,30.775612,-1.374109,0.0268,0.1037,0.0281,...,-2.336137,9.039455,2.336137,-9.025405,1.00,0.0,-30.0,30.0,175.0,1.0
8,8.0,4.0,9.0,-5.329708,-16.992711,5.388246,14.620758,0.0228,0.0883,0.0239,...,-2.741462,10.617154,2.741462,-10.605204,1.00,0.0,-30.0,30.0,175.0,1.0
9,9.0,5.0,9.0,-84.300792,-139.812332,85.237591,-122.710368,0.0139,0.0605,2.4590,...,-3.607148,15.700176,3.607148,-14.470676,1.00,0.0,-30.0,30.0,175.0,1.0
